In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words= VOCAB_SIZE)

In [7]:
len(train_data[1])

189

In [8]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

In [13]:
history = model.fit(
    train_data, 
    train_labels, 
    epochs=10,
    validation_split=0.2
)

Epoch 1/10
625/625 [==============================] - 19s 31ms/step - loss: 0.4276 - accuracy: 0.8065 - val_loss: 0.3558 - val_accuracy: 0.8706
Epoch 2/10
625/625 [==============================] - 19s 30ms/step - loss: 0.2380 - accuracy: 0.9113 - val_loss: 0.2687 - val_accuracy: 0.8900
Epoch 3/10
625/625 [==============================] - 19s 30ms/step - loss: 0.1877 - accuracy: 0.9309 - val_loss: 0.3064 - val_accuracy: 0.8952
Epoch 4/10
625/625 [==============================] - 19s 30ms/step - loss: 0.1547 - accuracy: 0.9453 - val_loss: 0.3151 - val_accuracy: 0.8754
Epoch 5/10
625/625 [==============================] - 19s 30ms/step - loss: 0.1302 - accuracy: 0.9555 - val_loss: 0.4242 - val_accuracy: 0.8658
Epoch 6/10
625/625 [==============================] - 19s 30ms/step - loss: 0.1121 - accuracy: 0.9621 - val_loss: 0.3199 - val_accuracy: 0.8880
Epoch 7/10
625/625 [==============================] - 19s 30ms/step - loss: 0.0965 - accuracy: 0.9671 - val_loss: 0.3055 - val_accuracy:

In [14]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 8s 11ms/step - loss: 0.5374 - accuracy: 0.8457
[0.5374112725257874, 0.8457199931144714]


In [15]:
word_index = imdb.get_word_index()

def encode_text(text):
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

1646592/1641221 [==============================] - 0s 0us/step


In [16]:
text = "that movie was just amazing, so amazing"
encoding = encode_text(text)
print(encoding)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [18]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
        if num != PAD:
            text += reverse_word_index[num] + " "
        
    return text[:-1]

print(decode_integers(encoding))
    

that movie was just amazing so amazing


In [22]:
def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1, 250))
    pred[0] = encoded_text
    result = model.predict(pred)
    print(result[0])

positive_review = "That movie was so awesome! I really loved it and would watch it again because it was so great"
predict(positive_review)

negative_review = "That movie sucked. I hated it and wouldn't watch it again."
predict(negative_review)

[0.58615786]
[0.758527]


# RNN Play Generator

In [23]:
from keras.preprocessing import sequence
import tensorflow as tf
import keras
import os
import numpy as np

In [24]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt',
                                        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [25]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [26]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [27]:
vocab = sorted(set(text))
# create a mapping
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [28]:
print("Text:", text[:13])
print("Encoding:", text_to_int(text[:13]))

Text: First Citizen
Encoding: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [29]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [30]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [31]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [32]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [33]:
for x, y in dataset.take(2):
    print("\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT\n")
    print(int_to_text(y))


EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT

irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 

EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT

re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [34]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [35]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform',
                             activation='tanh',
                             recurrent_activation='sigmoid',
                             recurrent_dropout=0),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [37]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [38]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 8.8019023e-04  3.2530690e-03 -3.2481849e-03 ... -1.7354025e-03
    6.6066235e-03  1.5243434e-03]
  [ 4.1613579e-03  2.8104195e-04 -5.7626478e-03 ...  1.7019592e-03
    9.3478998e-03  5.7835919e-03]
  [-2.5427202e-05  2.5477884e-03 -4.9664048e-03 ...  3.0876405e-04
    4.8196274e-03  5.2838326e-03]
  ...
  [-2.8815237e-03 -5.3799795e-03  1.3646820e-03 ... -4.5926883e-03
   -1.7348665e-03  1.1836044e-03]
  [-2.6868826e-03 -1.3835845e-03 -1.6558128e-03 ... -5.0750249e-03
    4.4914968e-03  2.7674329e-03]
  [-1.3152338e-03 -5.4534818e-03 -2.9774010e-04 ... -2.3216968e-03
    5.4914560e-03  3.8329919e-04]]

 [[-5.9693251e-03 -3.7929630e-03 -6.5869873e-04 ...  4.7229361e-03
   -4.9712318e-03  4.0759076e-03]
  [-3.9479095e-03 -7.7516483e-03 -6.6291653e-03 ...  7.1468926e-03
   -1.4603406e-04  2.4550508e-03]
  [ 3.6079099e-04 -8.1904903e-03 -8.4027939e-04 ...  1.0985746e-02
    2.8457176e-03 -4.2796024e-04]
  ...
  [ 4.3284362e-03  6.3339993e-03  2.6827678e-04 ...  4.7342577e

In [39]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[ 8.8019023e-04  3.2530690e-03 -3.2481849e-03 ... -1.7354025e-03
   6.6066235e-03  1.5243434e-03]
 [ 4.1613579e-03  2.8104195e-04 -5.7626478e-03 ...  1.7019592e-03
   9.3478998e-03  5.7835919e-03]
 [-2.5427202e-05  2.5477884e-03 -4.9664048e-03 ...  3.0876405e-04
   4.8196274e-03  5.2838326e-03]
 ...
 [-2.8815237e-03 -5.3799795e-03  1.3646820e-03 ... -4.5926883e-03
  -1.7348665e-03  1.1836044e-03]
 [-2.6868826e-03 -1.3835845e-03 -1.6558128e-03 ... -5.0750249e-03
   4.4914968e-03  2.7674329e-03]
 [-1.3152338e-03 -5.4534818e-03 -2.9774010e-04 ... -2.3216968e-03
   5.4914560e-03  3.8329919e-04]], shape=(100, 65), dtype=float32)


In [40]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[ 8.80190230e-04  3.25306901e-03 -3.24818492e-03  7.60958763e-04
 -4.88198409e-03  5.26125031e-03  5.57917450e-03 -4.24757542e-04
  2.06701946e-03 -1.46621803e-03  3.01002152e-03 -3.65632400e-03
 -3.25845880e-03  1.88306440e-03  1.20489276e-04  5.50129078e-03
 -6.77184016e-03  1.68993487e-04  4.11393400e-03 -4.77781170e-04
  1.12394616e-03 -1.92413083e-03 -1.61039445e-03 -3.30121373e-03
 -1.00579648e-03  7.70861236e-03 -4.61335003e-04  1.17314281e-04
  5.30266203e-03  2.19720270e-04 -3.04426439e-03  3.68219777e-03
  9.79654770e-03  7.21582095e-04 -2.55469070e-03 -2.45107105e-03
 -2.72494799e-04 -2.84120720e-03  3.96092003e-03  6.65480364e-03
  1.18306279e-03  7.32302759e-03 -1.64300518e-03  2.42383545e-03
  9.65212472e-03  4.74687607e-04  1.79817516e-03 -1.75667508e-03
  7.72410305e-04  4.43422515e-03 -3.34217073e-03  2.79051776e-04
  5.77395968e-03 -4.28572483e-03  4.47732222e-04 -2.63702660e-03
 -7.96843553e-04  4.11505718e-03 -2.00894987e-03  2.13737076e-05
 -4.3813204

In [42]:
sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

"sC-hnvnR?RMSIrYqsrl;MLHZ.poE&xXM$S!M.ffJWrZD$gSoQi!&gaRBWt:mxAFd,hTgIXUkuF'xxSk.GIrVOHUtEghnamFbH'sB"

In [43]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [44]:
model.compile('adam', loss=loss)

In [45]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [46]:
history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 [==============================] - 10s 56ms/step - loss: 2.5447
Epoch 2/40
172/172 [==============================] - 10s 57ms/step - loss: 1.8499
Epoch 3/40
172/172 [==============================] - 10s 57ms/step - loss: 1.6099
Epoch 4/40
172/172 [==============================] - 10s 57ms/step - loss: 1.4845
Epoch 5/40
172/172 [==============================] - 10s 58ms/step - loss: 1.4083
Epoch 6/40
172/172 [==============================] - 10s 58ms/step - loss: 1.3528
Epoch 7/40
172/172 [==============================] - 10s 58ms/step - loss: 1.3084
Epoch 8/40
172/172 [==============================] - 10s 58ms/step - loss: 1.2678
Epoch 9/40
172/172 [==============================] - 10s 59ms/step - loss: 1.2297
Epoch 10/40
172/172 [==============================] - 10s 58ms/step - loss: 1.1940
Epoch 11/40
172/172 [==============================] - 10s 58ms/step - loss: 1.1561
Epoch 12/40
172/172 [==============================] - 10s 58ms/step - loss: 1.1174
E

In [47]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [49]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [50]:
def generate_text(model, start_string):
    num_generate = 800

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)

        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

In [51]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

romeop her, out, a bannerd, in the virtues
Where the inf rouge must be a practise.
Who knew me an hundred with a soul of his action
At our beauty's sore so well they are fled,
As well the four access of death,
And not be bureful age, leanness, but the gires is but grows. Stand, fearful!
Thou hast deliver'd to he is yours;--
Wherein these blows being care to redeem me? so, if thou darest,
To God and man, you lie in vieward bill to reason by
And kiss her lip? and give our fathers,
Yet cannot name poison from sour mind: away!
Tell me, if once be full I joy,
Come, faints, a royal stats to rise;
Your fire-died, now thou lay and full of
great traffrow stices of what faults may call thee first and glodize
The knave my brother; therefore I compland;
For they one pleasant slander'd love.
Most migate mou


In [52]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________
